<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
import pandas as pd
import numpy as np

**1. Load and clean data**

In [0]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

In [0]:
votes = pd.read_csv('house-votes-84.data', 
                 header=None,
                 names=['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa'])

In [0]:
# CLEANING / REFORMAT DATA
votes = votes.replace({'?':np.NaN, 'n':0, 'y':1})
democrat = votes[votes['party'] == 'democrat']
republican = votes[votes['party'] == 'republican']

**2. Find an issue that democrats support more than republicans with p < 0.01**

In [0]:
from scipy.stats import ttest_1samp, ttest_ind

In [25]:
ttest_ind(democrat['anti-satellite-ban'], republican['anti-satellite-ban'], nan_policy='omit')

Ttest_indResult(statistic=12.526187929077842, pvalue=8.521033017443867e-31)

In [26]:
# EQUAL POPULATION VARIANCE QUESTION
ttest_ind(democrat['anti-satellite-ban'], republican['anti-satellite-ban'], 
          equal_var=False, nan_policy='omit')

Ttest_indResult(statistic=12.467492597567198, pvalue=1.362780681846568e-29)

This is testing the hypothesis that the difference between the average/expected values between democrats and republicans on 'anti-satellite-ban' equals 0.

*We reject the null hypothesis at a p-value = 0.01, with a p-value = 8.52 x 10<sup>-31</sup>*

Put another way, the chance of incorrectly rejecting the null hypothesis (Type 1 error) is: 8.52 x 10<sup>-31</sup>

**3. Find an issue that republicans support more than democrats with p < 0.01**

In [20]:
ttest_ind(republican['crime'], democrat['crime'], nan_policy='omit')

Ttest_indResult(statistic=16.342085656197696, pvalue=9.952342705606092e-47)

In [21]:
republican['crime'].mean()

0.9813664596273292

In [22]:
democrat['crime'].mean()

0.35019455252918286

This is testing the hypothesis that the difference between the average/expected values between republicans and democrats on 'crime' equals 0.

*We reject the null hypothesis at a p-value = 0.01, with a p-value = 9.95 x 10<sup>-47</sup>*

Put another way, the chance of incorrectly rejecting the null hypothesis (Type 1 error) is: 9.95 x 10<sup>-47</sup>

**4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)**

In [31]:
ttest_ind(republican['water-project'], democrat['water-project'], 
          nan_policy='omit')

Ttest_indResult(statistic=-0.08896538137868286, pvalue=0.9291556823993485)

This is testing the hypothesis that the difference between the average/expected values between republicans and democrats on 'water-project' equals 0.

*We fail to reject the null hypothesis at a p-value = 0.10, with a p-value = 0.929

Put another way, the chance of incorrectly rejecting the null hypothesis (Type 1 error) is: 0.929

**Stretch goals 1 - Refactor your code into functions so it's easy to rerun with arbitrary variables**

In [0]:
def two_sample_test(header, p_value_test):
  p_value = ttest_ind(republican[header], democrat[header], nan_policy='omit')
  print('P-Value: {}'.format(p_value[1]))
  if float(p_value_test) > p_value[1]:
    print('Reject the null hypothesis at p = {}'.format(p_value_test))  
  else:
    print('Fail to reject the null hypothesis at p = {}'.format(p_value_test))
  return float(p_value_test) > p_value[1]

In [63]:
two_sample_test('crime', 0.01)

P-Value: 9.952342705606092e-47
Reject the null hypothesis at p = 0.01


True

In [64]:
two_sample_test('water-project', 0.1)

P-Value: 0.9291556823993485
Fail to reject the null hypothesis at p = 0.1


False